# Fetch documents

In [42]:
import json
import requests

In [43]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [44]:
import minsearch

In [45]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [46]:
# with open('documents.json', 'rt') as f_in:
#     docs_raw = json.load(f_in)

In [47]:
# docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

In [48]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [49]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## index with minsearch

In [50]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [51]:
q = 'the course has already started, can I still enroll?'

In [52]:
index.fit(documents)

In [53]:
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [54]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## LLM response

In [56]:
from openai import OpenAI
client = OpenAI()
model = 'gpt-4o'

In [71]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {
            'role': 'user', 
            'content': q}
    ]
)
response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on several factors, including the institution's policies, the type of course, and how far into the course you are. Here are some steps you can take:\n\n1. **Check the Institution's Policy:** Look at the school or organization's website for any information regarding late enrollment or add/drop deadlines.\n\n2. **Contact the Instructor:** Reach out to the course instructor as they might have the authority to allow late enrollment, especially if you've missed only a small portion of the class.\n\n3. **Speak with an Academic Advisor:** Advisors can provide guidance on institutional policies and may help facilitate your late enrollment.\n\n4. **Consider Catch-up Work:** Be prepared to catch up on any missed lectures or assignments. Make sure you’re comfortable with the potential extra work this entails.\n\n5. **Evaluate Online or Recorded Options:** If the course is offered online or as a hybrid model with recorded lectu

In [61]:
prompt_template = """
You're a course teaching assistant.
Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the question.
If the CONTEXT doesn't contain the answer, output NONE.

Question: {question}

CONTEXT:
{context}
"""

In [68]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
context.strip()
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [72]:
prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

You're a course teaching assistant.
Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the question.
If the CONTEXT doesn't contain the answer, output NONE.

Question: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [73]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {
            'role': 'user', 
            'content': prompt}
    ]
)
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."